# Bands WorkGraph

## Introduction
In this tutorial, you will use `AiiDA-WorkGraph` to carry out a DFT calculation using Quantum ESPRESSO.

## Requirements
To run this tutorial, you need to install `aiida-quantumespresso` and `aiida-pseudo`. Open a terminal and run:

```console
pip install aiida-quantumespresso aiida-pseudo
aiida-pseudo install sssp -x PBEsol
```

Then, restart the AiiDA daemon.
```console
verdi daemon restart --reset
```


## Visualizing the WorkGraph

In [2]:
from workgraph_collections.qe.bands import bands_workgraph
from aiida import load_profile
load_profile()

wg = bands_workgraph()
wg.to_html()

### Prepare the inputs and submit the workflow


In [ ]:
from aiida import load_profile
from aiida.orm import Dict, KpointsData, StructureData, load_code, load_group
from ase.build import molecule

load_profile()
# create input structure node
mol = molecule("N2")
mol.center(vacuum=1.5)
mol.pbc = True
structure_n2 = StructureData(ase=mol)
# create the PW node
code = load_code("qe-7.2-pw@localhost")
paras = Dict({
        "CONTROL": {
            "calculation": "scf",
        },
        "SYSTEM": {
            "ecutwfc": 30,
            "ecutrho": 240,
            "occupations": "smearing",
            "smearing": "gaussian",
            "degauss": 0.1,
        },
    })
kpoints = KpointsData()
kpoints.set_kpoints_mesh([1, 1, 1])
# Load the pseudopotential family.
pseudo_family = load_group('SSSP/1.3/PBEsol/efficiency')
pseudos = pseudo_family.get_pseudos(structure=structure_n2)
#
metadata = {
    'options': {'resources': {
                'num_machines': 1,
                'num_mpiprocs_per_machine': 1,
            },
        }
    }

#------------------------- Set the inputs -------------------------
pw1.set({"code": code,
         "structure": structure_n2,
        "parameters": paras,
        "kpoints": kpoints,
        "pseudos": pseudos,
        "metadata": metadata
        })
#------------------------- Submit the calculation -------------------------
wg.submit(wait=True, timeout=200)
#------------------------- Print the output -------------------------
print('Energy of an un-relaxed N2 molecule: {:0.3f}'.format(pw1.outputs["output_parameters"].value.get_dict()["energy"]))


Generate node graph from the AiiDA process:

In [ ]:
from aiida_workgraph.utils import generate_node_graph
generate_node_graph(wg.pk)